In [2]:
import pyaudio
import wave
from datetime import datetime
from pydub import AudioSegment
from pathlib import Path
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import pyaudio
import wave
import time
import sys
sys.path.insert(0, str(Path.cwd().parent.parent))
import importlib
# 解决修改自定义包，无法生效的问题
from myutils import web_driver_manager

In [3]:
# 1、录音网上视频的声音
VIDEO_URL = 'https://www.youtube.com/watch?v=eyJ5kZeSF8k'
web_driver = web_driver_manager.get_driver_ChromeDriver(SetHeadless=False)
web_driver.implicitly_wait(30)
# 最大化窗口
web_driver.maximize_window()
# 打开视频
web_driver.get(VIDEO_URL)
vedio_area = web_driver.find_element(by=By.XPATH, value='//div[@id="player"]')
vedio_area.click()
hover = ActionChains(web_driver).move_to_element(
                vedio_area
            )
hover.perform()  # 悬停
duration_ele = web_driver.find_element(by=By.XPATH, value='//div[@class="ytp-time-display notranslate"]//span[@class="ytp-time-duration"]')
duration = duration_ele.text
print(duration)
# 等待视频开始播放
time.sleep(5)
duration = int(duration.split(":")[0])+1
print(duration)
# 初始化录音设备
p = pyaudio.PyAudio()
# 打开音频流
# 设置参数
CHUNK = 1024
FORMAT = pyaudio.paInt24
CHANNELS = 2
RATE = 48000
WAVE_OUTPUT_FILENAME = "output"
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
print("* recording")
# 开始录音
frames = []
for i in range(0, int(RATE / CHUNK * duration*60)):
    data = stream.read(CHUNK)
    frames.append(data)
print("* done recording")
# 停止录音
stream.stop_stream()
stream.close()
p.terminate()
web_driver.quit()
# 将音频数据保存为WAV文件
now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
wav_file_name = f'{WAVE_OUTPUT_FILENAME}_{now}.wav'
wf = wave.open(wav_file_name, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

8:31
9
* recording
* done recording
